In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
# pip install selenium-wire
# selenium-wire가 import 안되면 python -m pip install --upgrade pip를 실행하고 다시 설치한다.
from seleniumwire import webdriver # https://pypi.org/project/selenium-wire/
import json
import time

selenium-wire를 이용한 스타벅스 시도 코드 크롤링

In [13]:
option = Options()
option.add_argument('--window-size=1890,1030')
# driver = webdriver.Chrome('./chromedriver.exe', options=option) # 에러
driver = webdriver.Chrome(options=option)
driver.implicitly_wait(5)
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

request = driver.wait_for_request('https://www.starbucks.co.kr/store/getSidoList.do', timeout=10)
# print(request.url)
# print(request.response.status_code)
# print(request.response.headers)
# print(request.response.headers['Content-Type'])
# print(request.response.body.decode('utf-8'))
# print(type(request.response.body.decode('utf-8')))

starbucks = json.loads(request.response.body.decode('utf-8'))
# print(type(starbucks))
# print(starbucks)

sido_list = {}
for star in starbucks['list']:
    sido_list[star.get('sido_nm')] = star['sido_cd']
print(sido_list)

{'서울': '01', '경기': '08', '광주': '02', '대구': '03', '대전': '04', '부산': '05', '울산': '06', '인천': '07', '강원': '09', '경남': '10', '경북': '11', '전남': '12', '전북': '13', '충남': '14', '충북': '15', '제주': '16', '세종': '17'}


selenium-wire를 이용한 스타벅스 구군 코드 크롤링

In [20]:
option = Options()
option.add_argument('--window-size=1890,1030')
driver = webdriver.Chrome(options=option)
driver.implicitly_wait(5)
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

sido_list = {'서울': '01', '경기': '02', '광주': '03', '대구': '04', '대전': '05', '부산': '06', '울산': '07', 
             '인천': '08', '강원': '09', '경남': '10', '경북': '11', '전남': '12', '전북': '13', '충남': '14', 
             '충북': '15', '제주': '16', '세종': '17'}
sido = input('시도: ')

driver.find_element(By.XPATH,
    '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[' + 
                   str(int(sido_list[sido]))  + ']/a'
).click()
time.sleep(1)

request = driver.wait_for_request('https://www.starbucks.co.kr/store/getGugunList.do', timeout=10)
starbucks = json.loads(request.response.body.decode('utf-8'))
# print(starbucks)
gugun_list = {}
for star in starbucks['list']:
    gugun_list[star.get('gugun_nm')] = star['gugun_cd']
print(gugun_list)

시도: 서울
{'강남구': '0101', '강동구': '0120', '강북구': '0102', '강서구': '0103', '관악구': '0104', '광진구': '0105', '구로구': '0121', '금천구': '0106', '노원구': '0107', '도봉구': '0108', '동대문구': '0122', '동작구': '0109', '마포구': '0110', '서대문구': '0111', '서초구': '0112', '성동구': '0123', '성북구': '0113', '송파구': '0114', '양천구': '0115', '영등포구': '0116', '용산구': '0124', '은평구': '0117', '종로구': '0118', '중구': '0119', '중랑구': '0125'}


selenium-wire를 이용한 스타벅스 지역별 매장 정보 크롤링

In [31]:
option = Options()
option.add_argument('--window-size=1890,1030')
driver = webdriver.Chrome(options=option)
driver.implicitly_wait(5)
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

sido_list = {'서울': '01', '경기': '02', '광주': '03', '대구': '04', '대전': '05', '부산': '06', '울산': '07', 
             '인천': '08', '강원': '09', '경남': '10', '경북': '11', '전남': '12', '전북': '13', '충남': '14', 
             '충북': '15', '제주': '16', '세종': '17'}
sido = input('시도: ')
driver.find_element(By.XPATH,
    '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[' + 
                   str(int(sido_list[sido]))  + ']/a'
).click()
time.sleep(1)

request = driver.wait_for_request('https://www.starbucks.co.kr/store/getGugunList.do', timeout=10)
starbucks = json.loads(request.response.body.decode('utf-8'))
gugun_list = {}
for i, star in zip(range(2, len(starbucks['list']) + 2), starbucks['list']):
    gugun_list[star.get('gugun_nm')] = i
# print(gugun_list)

gugun = input('구군: ')
'''
driver.find_element(By.XPATH,
    '//*[@id="mCSB_2_container"]/ul/li[' + str(gugun_list[gugun]) +  ']/a'
).click()
'''
driver.find_element(By.CSS_SELECTOR,
    '#mCSB_2_container > ul > li:nth-child(' + str(gugun_list[gugun]) + ') > a '
).click()
time.sleep(1)

request = driver.wait_for_request('https://www.starbucks.co.kr/store/getStore.do', timeout=10)
starbucks = json.loads(request.response.body.decode('utf-8'))
store_list = {}
for i, star in zip(range(1, len(starbucks['list']) + 1), starbucks['list']):
    print('{:3d}. {}: {}({}, {})'.format(i, star['s_name'], star['doro_address'], star['lat'], star['lot']))

시도: 서울
구군: 서초구
  1. 서울웨이브아트센터: 서울특별시 서초구 잠원로 145-35 (잠원동)(37.518807, 127.00704)
  2. 서이초교사거리: 서울특별시 서초구 사임당로 143 (서초동)(37.4918146, 127.0252252)
  3. 강남역우송빌딩: 서울특별시 서초구 서초대로73길 7 (서초동)(37.4981314601413, 127.02527908458)
  4. 반포자이: 서울특별시 서초구 잠원로 24 (반포동)(37.507170475349405, 127.0099189083483)
  5. 강남에비뉴: 서울특별시 서초구 서초대로77길 62, 강남역아이파크 B102~B105 (서초동)(37.5027, 127.023894)
  6. 서울교대: 서울특별시 서초구 서초중앙로 64, 1층 (서초동)(37.487397, 127.015134)
  7. 서초동: 서울특별시 서초구 남부순환로339길 64, 현우빌딩 1층 (서초동)(37.485941, 127.021525)
  8. 강남GT타워: 서울특별시 서초구 서초대로 411, GT타워 (서초동)(37.498056, 127.02583300000003)
  9. 교대법원: 서울특별시 서초구 서초중앙로 164 (서초동)(37.496329, 127.013336)
 10. 서초역: 서울특별시 서초구 반포대로 118 (서초동)(37.49067495, 127.0086118)
 11. 몬테소리: 서울특별시 서초구 강남대로 399 (서초동)(37.4982695, 127.026865)
 12. 서래마을입구: 서울특별시 서초구 서래로 48 (반포동)(37.49925284, 126.9984801)
 13. 남부터미널2: 서울특별시 서초구 효령로 274 (서초동)(37.48436736, 127.0142192)
 14. 영동2교: 서울특별시 서초구 논현로 139, 대흥빌딩 (양재동)(37.48203954, 127.0425748)
 15. 신논현역: 서울특별시 서초구 강남대로 483 (반포동) 청호빌딩(37.505